In [1]:
import pandas as pd
import seaborn as sns

train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/train.csv")
test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/test.csv")

log_train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_train.csv")
log_test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_test.csv")

In [2]:
log_train_df= log_train
train_df = train

In [3]:
log_train_df['time'] = pd.to_datetime(log_train_df['time'])

# Extract features
log_train_df['hour'] = log_train_df['time'].dt.hour
log_train_df['day_of_week'] = log_train_df['time'].dt.dayofweek
log_train_df['day'] = log_train_df['time'].dt.day

# Count of events per ID
event_count = log_train_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_train_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
train_df_enriched = train_df.merge(event_count, on='ID', how='left')
train_df_enriched = train_df_enriched.merge(unique_sources, on='ID', how='left')


# Time difference features (e.g., average time between events for each ID)
log_train_df['time_diff'] = log_train_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_train_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')

# Fill missing values for IDs that did not have event data


train_df_enriched = train_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
train_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, "avg_time_diff":0}, inplace=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import ADASYN

# Assuming 'train_df_enriched' is your dataset
train_clean = train_df_enriched.dropna(subset=['label'])

X = train_clean.drop(['label', 'ID'], axis=1)
y = train_clean['label']

# Initialize the SimpleImputer with 'median' strategy
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Resampling
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2, random_state = 42)

# Define improved model parameters for 'dart' boosting
params = {
    'boosting_type': 'gbdt',  
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.01,
    'num_leaves': 300,
    'max_depth': -1,
    'min_child_samples': 30,
    'max_bin': 255,
    'subsample': 0.8,
    'subsample_freq': 1,
    'colsample_bytree': 0.8,
    'min_child_weight': 0.001,
    'subsample_for_bin': 200000,
    'min_split_gain': 0.0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'verbose': -1
}

# Creating the LightGBM data containers
dtrain = lgb.Dataset(X_train, label=y_train)
dtest = lgb.Dataset(X_test, label=y_test, reference=dtrain)

# Train the model
clf = lgb.train(params, dtrain, num_boost_round=2200, valid_sets=[dtrain, dtest],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])

# Predictions
predictions = clf.predict(X_test, num_iteration=clf.best_iteration)

# Evaluation
accuracy = accuracy_score(y_test, predictions.round())
print(f"Accuracy: {accuracy:.2f}")
print(confusion_matrix(y_test, predictions.round()))
print(classification_report(y_test, predictions.round()))


Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.340065	valid_1's binary_logloss: 0.354743
[200]	training's binary_logloss: 0.231144	valid_1's binary_logloss: 0.258733
[300]	training's binary_logloss: 0.183291	valid_1's binary_logloss: 0.225439
[400]	training's binary_logloss: 0.154296	valid_1's binary_logloss: 0.212362
[500]	training's binary_logloss: 0.132248	valid_1's binary_logloss: 0.206182
[600]	training's binary_logloss: 0.114604	valid_1's binary_logloss: 0.203174
[700]	training's binary_logloss: 0.100082	valid_1's binary_logloss: 0.201868
[800]	training's binary_logloss: 0.088054	valid_1's binary_logloss: 0.201209
[900]	training's binary_logloss: 0.0779067	valid_1's binary_logloss: 0.201196
Early stopping, best iteration is:
[852]	training's binary_logloss: 0.0825997	valid_1's binary_logloss: 0.201085
Accuracy: 0.92
[[12168  1414]
 [  613 12831]]
              precision    recall  f1-score   support

           0       0.95      0

In [11]:
from sklearn.metrics import f1_score

# Assuming `predictions` are the predicted probabilities for the positive class from your model
# And `y_test` are the true binary labels in the test set

# Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
binary_predictions = [1 if prob > 0.5 else 0 for prob in predictions]

# Calculate F1 score
f1 = f1_score(y_test, binary_predictions)

print(f'F1 Score: {f1}')
#F1_Score : 0.9221600231637873 
#F1_Score : 0.9229871645274212 - 0.1 median
#F1_Score : 0.9242930498300427 - mean lightbgm
#F1_Score : 0.9243412961382085 - median lightbgm
#F1_Score : 0.9246502317282795 - lightbgm median learning rate= 0.01
#F1_Score : 0.9207335755241388 - lightbgm median learning rate= 0.01 dart
#F1 Score: 0.9220892274211099 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100
#F1 Score: 0.924385564845984 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1400
#F1 Score: 0.9251454801749377 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1800
#F1 Score: 0.9258336042773021 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2200
#F1 Score: 0.9257118080647493 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2000
#F1 Score: 0.9261367730777545 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2000 - LR - 0.066
#F1 Score: 0.9253763595764605 - lightbgm median learning rate= 0.01 gbdt - with stopping_rounds=100 - num_boost_round=2000 - LR - 0.066
#F1 Score: 0.9266568058010751 - lightbgm median learning rate= 0.01 gbdt - with stopping_rounds=100 - num_boost_round=700 - LR - 0.066
#F1 Score: 0.9263788190311294 - lightbgm median learning rate= 0.01 gbdt - with stopping_rounds=100 - num_boost_round=700 - LR - 0.066 -num lev 50
#F1 Score: 0.9257345235083662 - lightbgm median lr= 0.01 dart - num_boost_round=2000 -50NL **
#F1 Score: 0.9256598187529336 - lightbgm median lr= 0.01 dart - num_boost_round=2400 -50NL 
#F1 Score: 0.9257774406761295 - lightbgm median lr= 0.01 dart - num_boost_round=2200 -50NL *** -> score: 0.87251
#F1 Score: 0.926345404703506 - lightbgm median lr= 0.01 dart - num_boost_round=2200 -50NL - imp_features
#F1 Score: 0.9256610316428262 - lightbgm median lr= 0.01 dart - num_boost_round=2200 -50NL *** -> score:0.87217
#F1 Score: 0.9254582190792322 - lightbgm median lr= 0.01 gbdt - num_boost_round=2200 -50NL *** -> score:0.87048
#F1 Score: 0.9267940337318069 - lightbgm median lr= 0.01 gbdt - num_boost_round=2200 -300NL *** -> score:0.87048

F1 Score: 0.9267940337318069


In [12]:
from sklearn.metrics import roc_auc_score

# Assuming `predictions` are the predicted probabilities for the positive class from your model
# And `y_test` are the true binary labels in the test set

# Calculate AUC score
auc_score = roc_auc_score(y_test, predictions)

print(f'AUC Score: {auc_score}')
#AUC Score: 0.9234141033971461 - lightbgm median learning rate= 0.01
#AUC Score: 0.9198248761840062 - lightbgm median learning rate= 0.01 dart
#AUC Score: 0.9646368563832868 - lightbgm median learning rate= 0.01 dart without fold
#AUC Score: 0.9645827863163663 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100
#AUC Score: 0.9659848237540357 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1400
#AUC Score: 0.9666748373275778 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1800
#AUC Score: 0.9670271115081299 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2200
#AUC Score: 0.9668365984505018 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2000
#AUC Score: 0.9670062293886964 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2000 - LR - 0.066
#AUC Score: 0.9667961485857925 - lightbgm median learning rate= 0.01 gbdt - with stopping_rounds=100 - num_boost_round=2000 - LR - 0.066
#AUC Score: 0.9677003777642768 - lightbgm median learning rate= 0.01 gbdt - with stopping_rounds=100 - num_boost_round=700 - LR - 0.066
#AUC Score: 0.9670806886847413 - lightbgm median learning rate= 0.01 gbdt - with stopping_rounds=100 - num_boost_round=700 - LR - 0.066 -num lev 50
#AUC Score: 0.9670328673716297 - lightbgm median lr= 0.01 dart - num_boost_round=2000 -50NL **
#AUC Score: 0.9672647174965239 - lightbgm median lr= 0.01 dart - num_boost_round=2400 -50NL 
#AUC Score: 0.9672058554404861 - lightbgm median lr= 0.01 dart - num_boost_round=2200 -50NL ***
#AUC Score: 0.9698888726076903 - lightbgm median lr= 0.01 dart - num_boost_round=2200 -50NL - imp_features
#AUC Score: 0.9671630177960566 - lightbgm median lr= 0.01 dart - num_boost_round=2200 -50NL *** -> score:0.87217
#AUC Score: 0.9674264074241812 - lightbgm median lr= 0.01 gbdt - num_boost_round=2200 -50NL *** -> score:0.87048
#AUC Score: 0.9686672259182667 - lightbgm median lr= 0.01 gbdt - num_boost_round=2200 -300NL *** -> score:

AUC Score: 0.9686672259182667


In [7]:
log_test_df=log_test
test_df = test

In [8]:
log_test_df['time'] = pd.to_datetime(log_test_df['time'])

# Extract features
log_test_df['hour'] = log_test_df['time'].dt.hour
log_test_df['day_of_week'] = log_test_df['time'].dt.dayofweek
log_test_df['day'] = log_test_df['time'].dt.day

# Count of events per ID
event_count = log_test_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_test_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
test_df_enriched = test_df.merge(event_count, on='ID', how='left')
test_df_enriched = test_df_enriched.merge(unique_sources, on='ID', how='left')



# Time difference features (e.g., average time between events for each ID)
log_test_df['time_diff'] = log_test_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_test_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')



test_df_enriched = test_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
test_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, 'avg_time_diff':0}, inplace=True)

In [13]:
# Assuming `test_df_enriched` is your test dataset and `clf` is your trained LightGBM model

# Convert the 'ID' column from float to int
test_df_enriched['ID'] = test_df_enriched['ID'].astype(int)

# Separate out the 'ID' column and features from the 'test' dataset
#test_features = test_df_enriched.drop('ID', axis=1)
test_features = test_df_enriched.drop('ID', axis=1)

# Impute the 'test' dataset using the fitted SimpleImputer
# Note: We use .transform() here, NOT .fit_transform()
test_imputed = imputer.transform(test_features)

# Predict the labels using the trained LightGBM model
predicted_labels = clf.predict(test_imputed)

# For binary classification, LightGBM might output probabilities. If so, convert these to 0 or 1
# Uncomment the following line if your model outputs probabilities
predicted_labels = np.round(predicted_labels).astype(int)

# Create a DataFrame with 'ID' and the predicted 'label'
results_df = pd.DataFrame({
    'ID': test_df_enriched['ID'],
    'LABEL': predicted_labels
})

# Optionally, convert the 'LABEL' column to the desired format, e.g., int
# results_df['LABEL'] = results_df['LABEL'].astype(int)

# Output the DataFrame with 'ID' and 'label'
# Save to a CSV file
results_df.to_csv('/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/Submissions/my_submission_39_lGBM_ADAS_mdn_0.01LR_bgdt_wo_fold_2200BR_300NL.csv', index=False)


# Display the first few rows of the DataFrame
print(results_df.head())


       ID  LABEL
0   17547      0
1  140449      1
2  182658      1
3  149652      1
4  106304      1
